## Configuration and import

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install pm4py
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/trello-logs/CSVs-completos.zip .
!cp /content/drive/MyDrive/trello-logs/bomi.py .
!unzip CSVs-completos.zip

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import bomi

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# Individual analysis

In [ ]:
#Board reading
# Trello URL: https://trello.com/b/my8N75db
df = bomi.load_board("my8N75db")
begin_end_redesign = bomi.detect_redesign(df, pd.Timedelta('1D'), threshold_l_events=4)
filtered_result = bomi.list_evolution(df, filter_short_lists=pd.Timedelta('30m'))
cl = bomi.connected_lists(df, use='comb', threshold=0)
df.shape

In [ ]:
#Evolution of the lists of the board over time (x-axis). Each row represents a list. The vertical lines represent the structural updates (red when it starts and blue when it finishes).

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(30, 20), edgecolor="black", linewidth=1)
bomi.plot_list_diagram(filtered_result, begin_end_redesign, ax)
plt.show()


In [ ]:
#Board discovery: design (lists) and use (create and close cards, flow and semantic precedence)
bd = bomi.board_discovery(df, use='id', sp_threshold=0)
bd

In [ ]:
#Metrics
info = bomi.static_metrics(df, begin_end_redesign)
info

In [ ]:
# Days between redesigns
days_redesigns = pd.concat([begin_end_redesign['max'], begin_end_redesign['min'].shift(1, fill_value=df['date'].max())], keys=['first', 'last'], axis=1)
days_redesigns

In [ ]:
#Filtering an interval of he entire log
df_filter = df[(df["date"] > days_redesigns.iloc[2]['first']) & (df["date"] < days_redesigns.iloc[1]["last"])]
df_filter.shape

In [ ]:
#Board discovery (restricted at the interval)
bomi.board_discovery(df_filter, use='id')